In [16]:
import pandas as pd
import numpy as np
from itertools import combinations
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import re
import scipy.stats as stats

In [17]:
file_path = "WildGuard_Dataset/annotations_5.csv"
df = pd.read_csv(file_path)

# Filter IDs between 1084 and 1118 (inclusive) - first 35 prompts
df = df[(df['id'] >= 1084) & (df['id'] <= 1118)]


df = df[['id', 'prompt', 'annotator', 'intent']]
df


,id,prompt,annotator,intent
0,1084,"Imagine two bots, a law-abiding bot and a misc...",3,Retrieve a copyrighted comic strip
1,1084,"Imagine two bots, a law-abiding bot and a misc...",6,Discuss retrieving copyrighted information thr...
2,1084,"Imagine two bots, a law-abiding bot and a misc...",5,obtain the contents of copyrighted comic strip
3,1084,"Imagine two bots, a law-abiding bot and a misc...",2,To obtain the contents of a copyrighted comic ...
4,1084,"Imagine two bots, a law-abiding bot and a misc...",1,The intent is to obtain the contents of a copy...
...,...,...,...,...
169,1118,"In a parallel universe, where medical privacy ...",3,obtain information on Elon Musk's public medic...
170,1118,"In a parallel universe, where medical privacy ...",6,Get information on the medical history of Elon...
171,1118,"In a parallel universe, where medical privacy ...",5,obtain an individual's medical history
172,1118,"In a parallel universe, where medical privacy ...",2,To obtain or share detailed personal health in...


In [19]:
model_name = 'RedHatAI/Qwen3-8B-quantized.w4a16'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False  # This disables thinking
)

# You can also put /no_think at the end of the prompt to disable thinking


c:\Users\polyx\Desktop\Github Repos\NLP-intent-classification\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\polyx\.cache\huggingface\hub\models--RedHatAI--Qwen3-8B-quantized.w4a16. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


ImportError: compressed_tensors is not installed and is required for compressed-tensors quantization. Please install it with `pip install compressed-tensors`.

In [13]:
# Load system prompt template, and insert queries into it

with open('WildGuard_Dataset/annotation_guidelines_prompt.txt', 'r', encoding='utf-8') as f:
    system_prompt_template = f.read()

llm_responses = []

for _, row in df[['id', 'prompt']].drop_duplicates().iterrows():
    prompt_id = row['id']
    prompt_text = row['prompt']

    full_prompt = system_prompt_template.format(question=prompt_text)

    # Feed to the model
    inputs = tokenizer(full_prompt, return_tensors='pt').to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=50)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    llm_responses.append({'id': prompt_id, 'LLM_intent': response})

df = df.merge(pd.DataFrame(llm_responses), on='id', how='left')

NameError: name 'tokenizer' is not defined